In [1]:
# Dependencies
import pandas as pd
import numpy as np

In [2]:
# Read imdb dataset in csv format
movies_df = pd.read_csv('resources/IMDb_movies.csv', encoding='utf8')

C:\Users\gucas\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Filter columns to keep from dataset
cleaned_movies_df = movies_df.drop(columns=['date_published', 'description', 'original_title', 'duration', 'language', 'actors', 'avg_vote', 'votes', 'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore', 'reviews_from_users', 'reviews_from_critics'])
cleaned_movies_df.head()

,imdb_title_id,title,year,genre,country,director,writer,production_company
0,tt0000009,Miss Jerry,1894,Romance,USA,Alexander Black,Alexander Black,Alexander Black Photoplays
1,tt0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",Australia,Charles Tait,Charles Tait,J. and N. Tait
2,tt0001892,Den sorte drøm,1911,Drama,"Germany, Denmark",Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama
3,tt0002101,Cleopatra,1912,"Drama, History",USA,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players
4,tt0002130,L'Inferno,1911,"Adventure, Drama, Fantasy",Italy,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film


In [4]:
# Conver tuple in multiple value director column to list
cleaned_movies_df = cleaned_movies_df.assign(director=cleaned_movies_df.director.str.split(','))
cleaned_movies_df.head(6)

,imdb_title_id,title,year,genre,country,director,writer,production_company
0,tt0000009,Miss Jerry,1894,Romance,USA,[Alexander Black],Alexander Black,Alexander Black Photoplays
1,tt0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",Australia,[Charles Tait],Charles Tait,J. and N. Tait
2,tt0001892,Den sorte drøm,1911,Drama,"Germany, Denmark",[Urban Gad],"Urban Gad, Gebhard Schätzler-Perasini",Fotorama
3,tt0002101,Cleopatra,1912,"Drama, History",USA,[Charles L. Gaskill],Victorien Sardou,Helen Gardner Picture Players
4,tt0002130,L'Inferno,1911,"Adventure, Drama, Fantasy",Italy,"[Francesco Bertolini, Adolfo Padovan]",Dante Alighieri,Milano Film
5,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...",1912,"Biography, Drama",USA,[Sidney Olcott],Gene Gauntier,Kalem Company


In [5]:
# Using Pandas explode to split director names into another column
cleaned_movies_df = cleaned_movies_df.explode('director')
print(len(cleaned_movies_df))
cleaned_movies_df.head(6)

91681


,imdb_title_id,title,year,genre,country,director,writer,production_company
0,tt0000009,Miss Jerry,1894,Romance,USA,Alexander Black,Alexander Black,Alexander Black Photoplays
1,tt0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",Australia,Charles Tait,Charles Tait,J. and N. Tait
2,tt0001892,Den sorte drøm,1911,Drama,"Germany, Denmark",Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama
3,tt0002101,Cleopatra,1912,"Drama, History",USA,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players
4,tt0002130,L'Inferno,1911,"Adventure, Drama, Fantasy",Italy,Francesco Bertolini,Dante Alighieri,Milano Film
4,tt0002130,L'Inferno,1911,"Adventure, Drama, Fantasy",Italy,Adolfo Padovan,Dante Alighieri,Milano Film


In [6]:
# Drop rows with missing data
cleaned_movies_df = cleaned_movies_df.dropna(how='any')
print(len(cleaned_movies_df))
cleaned_movies_df.head(6)

85562


,imdb_title_id,title,year,genre,country,director,writer,production_company
0,tt0000009,Miss Jerry,1894,Romance,USA,Alexander Black,Alexander Black,Alexander Black Photoplays
1,tt0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",Australia,Charles Tait,Charles Tait,J. and N. Tait
2,tt0001892,Den sorte drøm,1911,Drama,"Germany, Denmark",Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama
3,tt0002101,Cleopatra,1912,"Drama, History",USA,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players
4,tt0002130,L'Inferno,1911,"Adventure, Drama, Fantasy",Italy,Francesco Bertolini,Dante Alighieri,Milano Film
4,tt0002130,L'Inferno,1911,"Adventure, Drama, Fantasy",Italy,Adolfo Padovan,Dante Alighieri,Milano Film


In [7]:
# Drop duplicates title_id (generated after splitting directors) to keep only one director name
cleaned_movies_df = cleaned_movies_df.drop_duplicates(subset ='imdb_title_id', keep = 'first')

In [8]:
# Reset index (to avoid assigning the same director id to recently split director name since index will be used
# as a positional argument below)
cleaned_movies_df = cleaned_movies_df.reset_index(drop=True)
cleaned_movies_df.head()

,imdb_title_id,title,year,genre,country,director,writer,production_company
0,tt0000009,Miss Jerry,1894,Romance,USA,Alexander Black,Alexander Black,Alexander Black Photoplays
1,tt0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",Australia,Charles Tait,Charles Tait,J. and N. Tait
2,tt0001892,Den sorte drøm,1911,Drama,"Germany, Denmark",Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama
3,tt0002101,Cleopatra,1912,"Drama, History",USA,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players
4,tt0002130,L'Inferno,1911,"Adventure, Drama, Fantasy",Italy,Francesco Bertolini,Dante Alighieri,Milano Film


In [9]:
# Add director_id to directors
count=0
cleaned_movies_df['director_id']=""
for index, name in cleaned_movies_df.iterrows():
    cleaned_movies_df.loc[index, 'director_id']=count
    count+=1
cleaned_movies_df.head()

,imdb_title_id,title,year,genre,country,director,writer,production_company,director_id
0,tt0000009,Miss Jerry,1894,Romance,USA,Alexander Black,Alexander Black,Alexander Black Photoplays,0
1,tt0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",Australia,Charles Tait,Charles Tait,J. and N. Tait,1
2,tt0001892,Den sorte drøm,1911,Drama,"Germany, Denmark",Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,2
3,tt0002101,Cleopatra,1912,"Drama, History",USA,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,3
4,tt0002130,L'Inferno,1911,"Adventure, Drama, Fantasy",Italy,Francesco Bertolini,Dante Alighieri,Milano Film,4


In [10]:
# Find the issue in year column (this a specific issue related to this dataset)
cleaned_movies_df[cleaned_movies_df['year']=='TV Movie 2019']

,imdb_title_id,title,year,genre,country,director,writer,production_company,director_id
78492,tt8206668,Bad Education,TV Movie 2019,"Biography, Comedy, Crime",USA,Cory Finley,"Mike Makowsky, Robert Kolker",HBO Films,78492


In [11]:
# Replaced a row in year that has a value of TV Movie 2019 for just the year
for index, row in cleaned_movies_df.iterrows():
    a = len(str(row['year']))
    if a > 4 :
            cleaned_movies_df.loc[index, 'year'] = row['year'][-4:]

In [12]:
# Verify that the row value changed to just the year (using the movie id since the year value changed)
cleaned_movies_df[cleaned_movies_df['imdb_title_id']=='tt8206668']

,imdb_title_id,title,year,genre,country,director,writer,production_company,director_id
78492,tt8206668,Bad Education,2019,"Biography, Comedy, Crime",USA,Cory Finley,"Mike Makowsky, Robert Kolker",HBO Films,78492


In [13]:
# Read oscars awards dataset in csv format
oscars_df = pd.read_csv('resources/the_oscar_award.csv', encoding='utf8')
print(len(oscars_df))
oscars_df.head()

10395


,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


In [14]:
# Drop rows with missing data
oscars_df = oscars_df.dropna(how='any')
print(len(oscars_df))
oscars_df.head(6)

10091


,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False
5,1927,1928,1,ART DIRECTION,Rochus Gliese,Sunrise,False


In [15]:
# Keep only a selected number of categories
cleaned_oscars_df = oscars_df[(oscars_df['winner']==True) &
                              ((oscars_df['category']=='BEST PICTURE') | 
                               (oscars_df['category']=='DIRECTING') | 
                               (oscars_df['category']=='BEST MOTION PICTURE') | 
                               (oscars_df['category']=='CINEMATOGRAPHY (Black-and-White)') |
                               (oscars_df['category']=='CINEMATOGRAPHY (Color)'))]
print(len(cleaned_oscars_df))
cleaned_oscars_df.head()

221


,year_film,year_ceremony,ceremony,category,name,film,winner
59,1928,1929,2,DIRECTING,Frank Lloyd,The Divine Lady,True
98,1929,1930,3,DIRECTING,Lewis Milestone,All Quiet on the Western Front,True
139,1930,1931,4,DIRECTING,Norman Taurog,Skippy,True
171,1931,1932,5,DIRECTING,Frank Borzage,Bad Girl,True
234,1932,1933,6,DIRECTING,Frank Lloyd,Cavalcade,True


In [16]:
# Conver tuple in multiple value director column to list for selected categories above
# Every award category has different formatting to list the names of winners, need to filter by group categories to properly
# clean and split the names
cleaned_oscars_df = cleaned_oscars_df.assign(name= cleaned_oscars_df.name.str.split('and'))
print(len(cleaned_oscars_df))
cleaned_oscars_df.head()

221


,year_film,year_ceremony,ceremony,category,name,film,winner
59,1928,1929,2,DIRECTING,[Frank Lloyd],The Divine Lady,True
98,1929,1930,3,DIRECTING,[Lewis Milestone],All Quiet on the Western Front,True
139,1930,1931,4,DIRECTING,[Norman Taurog],Skippy,True
171,1931,1932,5,DIRECTING,[Frank Borzage],Bad Girl,True
234,1932,1933,6,DIRECTING,[Frank Lloyd],Cavalcade,True


In [17]:
# Using Pandas explode to split awarded director names into another column
cleaned_oscars_df = cleaned_oscars_df.explode('name')
print(len(cleaned_oscars_df))
cleaned_oscars_df.head(6)

257


,year_film,year_ceremony,ceremony,category,name,film,winner
59,1928,1929,2,DIRECTING,Frank Lloyd,The Divine Lady,True
98,1929,1930,3,DIRECTING,Lewis Milestone,All Quiet on the Western Front,True
139,1930,1931,4,DIRECTING,Norman Taurog,Skippy,True
171,1931,1932,5,DIRECTING,Frank Borzage,Bad Girl,True
234,1932,1933,6,DIRECTING,Frank Lloyd,Cavalcade,True
282,1934,1935,7,DIRECTING,Frank Capra,It Happened One Night,True


In [18]:
# Drop duplicates indices (generated after splitting name) to keep only one director name
cleaned_oscars_df = cleaned_oscars_df[~cleaned_oscars_df.index.duplicated(keep='first')]
print(len(cleaned_oscars_df))

221


In [19]:
# Reset index
cleaned_oscars_df = cleaned_oscars_df.reset_index(drop=True)
cleaned_oscars_df.head(6)

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1928,1929,2,DIRECTING,Frank Lloyd,The Divine Lady,True
1,1929,1930,3,DIRECTING,Lewis Milestone,All Quiet on the Western Front,True
2,1930,1931,4,DIRECTING,Norman Taurog,Skippy,True
3,1931,1932,5,DIRECTING,Frank Borzage,Bad Girl,True
4,1932,1933,6,DIRECTING,Frank Lloyd,Cavalcade,True
5,1934,1935,7,DIRECTING,Frank Capra,It Happened One Night,True


In [20]:
# Add movie ID to oscars movies dataframe
cleaned_oscars_df['title_id']=""

for index, film in cleaned_oscars_df.iterrows():
    for index2, title in cleaned_movies_df.iterrows():
            if (str(film['film']).lower() == str(title['title']).lower()):
                cleaned_oscars_df.loc[index, 'title_id'] = title['imdb_title_id']
                
print(len(cleaned_oscars_df))
cleaned_oscars_df.head(6)

221


,year_film,year_ceremony,ceremony,category,name,film,winner,title_id
0,1928,1929,2,DIRECTING,Frank Lloyd,The Divine Lady,True,
1,1929,1930,3,DIRECTING,Lewis Milestone,All Quiet on the Western Front,True,
2,1930,1931,4,DIRECTING,Norman Taurog,Skippy,True,tt0163246
3,1931,1932,5,DIRECTING,Frank Borzage,Bad Girl,True,tt3598108
4,1932,1933,6,DIRECTING,Frank Lloyd,Cavalcade,True,tt0427047
5,1934,1935,7,DIRECTING,Frank Capra,It Happened One Night,True,


In [21]:
# Dropped rows where the awarded movies are not listed in the imdb dataset (empty rows)
cleaned2_oscars_df = cleaned_oscars_df.replace('', np.NaN)
cleaned2_oscars_df = cleaned2_oscars_df.dropna(how='any')

# Reset index
cleaned2_oscars_df = cleaned2_oscars_df.reset_index(drop=True)

print(len(cleaned2_oscars_df))
cleaned2_oscars_df.head(6)

74


,year_film,year_ceremony,ceremony,category,name,film,winner,title_id
0,1930,1931,4,DIRECTING,Norman Taurog,Skippy,True,tt0163246
1,1931,1932,5,DIRECTING,Frank Borzage,Bad Girl,True,tt3598108
2,1932,1933,6,DIRECTING,Frank Lloyd,Cavalcade,True,tt0427047
3,1935,1936,8,DIRECTING,John Ford,The Informer,True,tt0020025
4,1943,1944,16,DIRECTING,Michael Curtiz,Casablanca,True,tt10497826
5,1944,1945,17,CINEMATOGRAPHY (Black-and-White),Joseph LaShelle,Laura,True,tt0093351


In [22]:
# Export Final cleaned and filtered IMdb dataset in csv format
cleaned_movies_df.to_csv('output/cleaned_imdb.csv')

# Export oscars cleaned and filtered dataframe
cleaned2_oscars_df.to_csv('output/final_oscars.csv')

In [23]:
# Filter director name and id from the imdb dataset
cleaned_directors = cleaned_movies_df[['director', 'director_id']]
print(len(cleaned_directors))
cleaned_directors.head(6)

80184


,director,director_id
0,Alexander Black,0
1,Charles Tait,1
2,Urban Gad,2
3,Charles L. Gaskill,3
4,Francesco Bertolini,4
5,Sidney Olcott,5


In [24]:
# Export directors dataframe to csv
cleaned_directors.to_csv('output/directors.csv')

In [25]:
# Filter unique Oscars categories from selected ones above
cleaned_categories_df = pd.DataFrame(cleaned_oscars_df['category'].unique())
cleaned_categories_df = cleaned_categories_df.rename(columns={0:'categories'})

# Add oscars_id to categories
count=0
cleaned_categories_df['id']=""
for index, categories in cleaned_categories_df.iterrows():
    cleaned_categories_df.loc[index, 'id']=count
    count+=1
    
cleaned_categories_df.head(6)

,categories,id
0,DIRECTING,0
1,CINEMATOGRAPHY (Black-and-White),1
2,CINEMATOGRAPHY (Color),2
3,BEST MOTION PICTURE,3
4,BEST PICTURE,4


In [26]:
# Export oscars categories
cleaned_categories_df.to_csv('output/oscars_categories.csv')

In [27]:
# Assign Oscars id to their respective awarded movie title
cleaned2_oscars_df['oscar_id'] = ""

for index, movies in cleaned2_oscars_df.iterrows():
    if movies['category'] == 'DIRECTING':
        cleaned2_oscars_df.loc[index, 'oscar_id'] = 0
    elif movies['category'] == 'CINEMATOGRAPHY (Black-and-White)':
        cleaned2_oscars_df.loc[index, 'oscar_id'] = 1
    elif movies['category'] == 'CINEMATOGRAPHY (Color)':
        cleaned2_oscars_df.loc[index, 'oscar_id'] = 2
    elif movies['category'] == 'BEST MOTION PICTURE':
        cleaned2_oscars_df.loc[index, 'oscar_id'] = 3
    elif movies['category'] == 'BEST PICTURE':
        cleaned2_oscars_df.loc[index, 'oscar_id'] = 4

        
# Filter to only keep movie awards
movie_awards_df = cleaned2_oscars_df[
                              ((cleaned2_oscars_df['category']=='BEST PICTURE') | 
                               (cleaned2_oscars_df['category']=='BEST MOTION PICTURE') | 
                               (cleaned2_oscars_df['category']=='CINEMATOGRAPHY (Black-and-White)') |
                               (cleaned2_oscars_df['category']=='CINEMATOGRAPHY (Color)'))]
     
# Only keep title_id and oscar_id relationship dataset
movie_awards_df = movie_awards_df[['title_id', 'oscar_id']]

# Reset index
movie_awards_df = movie_awards_df.reset_index(drop=True)

movie_awards_df.head(6)

,title_id,oscar_id
0,tt0093351,1
1,tt1781058,2
2,tt0435669,1
3,tt0026402,3
4,tt0116477,3
5,tt0029081,2


In [28]:
# Export relationship of title_id with oscar_id
movie_awards_df.to_csv('output/movie_awards.csv')

In [29]:
# Filter to only keep directing awards (it reduces looping time by only looping over the directors that have won directing
# rather than looping over all the directors listed on the movie awards)
directing_awards_df = cleaned2_oscars_df[(cleaned2_oscars_df['category']=='DIRECTING')]

# Reset index
directing_awards_df = directing_awards_df.reset_index(drop=True)

# Creating empty director_id column
directing_awards_df['director_id'] = ""

# Assign Director id to their respective awarded director name
for index, director in directing_awards_df.iterrows():
    for index2, reference in cleaned_directors.iterrows():
        if str(director['name']) == str(reference['director']):
            dir_id = cleaned_directors.at[index2, 'director_id']
            directing_awards_df.at[index, 'director_id'] = dir_id
            
# Only keep title_id and oscar_id relationship dataset
directing_awards_df = directing_awards_df[['oscar_id', 'director_id']]

print(len(directing_awards_df))
directing_awards_df

34


,oscar_id,director_id
0,0,13608
1,0,9795
2,0,8333
3,0,12401
4,0,10621
5,0,17299
6,0,17335
7,0,14405
8,0,
9,0,27149


In [30]:
# Drop not found directors in the main imdb dataset
directing_awards_df = directing_awards_df.replace('', 'NaN')
directing_awards_df = directing_awards_df[directing_awards_df['director_id'] != 'NaN']
directing_awards_df = directing_awards_df.drop_duplicates(subset ='director_id', keep = 'first')
# directing_awards_df['director_id'] = directing_awards_df['director_id'].astype(int)

# Reset index
directing_awards_df = directing_awards_df.reset_index(drop=True)

print(len(directing_awards_df))
directing_awards_df

31


,oscar_id,director_id
0,0,13608
1,0,9795
2,0,8333
3,0,12401
4,0,10621
5,0,17299
6,0,17335
7,0,14405
8,0,27149
9,0,19760


In [31]:
# Export relationship of oscar_id with director_id
directing_awards_df.to_csv('output/directing_awards.csv')